## Estruturas de delay

O efeito de atraso, em inglês efeito de delay, é uma das técnicas mais simples, porém extremamente valiosa quando utilizada de forma adequada. O processo de aplicação de estruturas de delay envolve pegar um sinal de áudio, armazená-lo temporariamente na memória e, em seguida, reproduzi-lo após um determinado período de tempo.

Nessa subseção, serão mostradas duas estruturas de delay: o filtro \ac{FIR} comb e o filtro \ac{IIR} comb. O efeito Comb Filter acontece quando dois sons idênticos são reproduzidos simultaneamente, porém com um atraso entre eles. O nome "Comb Filter" foi dado porque, quando visualizado em um analisador de espectro, o resultado se assemelha a um pente de cabelo, comb em inglês. Esse intervalo de atraso pode abranger desde um único valor de amostra até vários milissegundos, chegando a cerca de 15ms-20ms. No entanto, uma vez que o atraso ultrapassa esse limite, o ouvido humano começa a percebê-lo como um atraso distinto.


Importaremos todas as funções apresentadas no capítulo de introdução:

In [1]:
%run -i 1introducao.ipynb

##### FIR comb 
Um FIR comb, também chamado de forma feedfoward, é um filtro que simula um único atraso. A resposta temporal desse filtro consiste na combinação do sinal direto com uma versão atrasada.

<center><p>
<img src= "https://upload.wikimedia.org/wikipedia/commons/6/68/Comb_filter_feedforward.png" width=400px>
</p>
<p>
    <font size=2><em>Figura 1 - Imagem de <a href="https://commons.wikimedia.org/wiki/File:Comb_filter_feedforward.png">Wikimedia commons</a></em></font>
</p></center>


Dentro do filtro FIR comb, temos:

- **gain**: ou frequência de corte se refere à frequência na qual o equalizador começa a afetar o sinal de áudio, seja amplificando, atenuando ou filtrando determinadas faixas de frequência.
- **delay_length**: define se o filtro será low shelving ou high shelving.

Abaixo a função que definirá nosso filtro shelving.

In [2]:
def fircomb(input_signal, gain, delay_length):
    # inicialização do vetor de saída
    y = [0] * len(input_signal)
    
    # inicialização do vetor de delay
    delay_line = np.zeros(delay_length)

    # aplicação do delay
    for n in range(len(input_signal)):
        y[n] = input_signal[n] + gain * delay_line[delay_length - 1]
        delay_line[1:] = delay_line[:-1]
        delay_line[0] = input_signal[n]
        
    # calcula a resposta em frequência do filtro.
    w, h = freqz(y, input_signal, fs=44100)

    return y, w, h


##### FIR comb 
Ao contrário do filtro FIR comb, como o nome diz, o filtro IIR comb produz uma série infinita de respostas para um dado sinal de entrada. Este filtro, que também pode ser chamado de forma feedback, circula o sinal de entrada em uma linha de atraso que é realimentada de volta para a entrada. A cada ciclo na linha de atraso, o sinal é atenuado por um fator de ganho $g$ através de uma amplificação intensa causada pela estrutura do filtro. Devido ao laço de realimentação, a resposta temporal do filtro é infinita. 

As principais diferenças entre o filtro IIR comb e o filtro FIR comb são que o ganho aumenta consideravelmente e os picos de frequência se tornam mais estreitos à medida que $|g|$ se aproxima de 1, isso mostra um filtro mais "forte" que o FIR comb.

<center><p>
<img src= "https://upload.wikimedia.org/wikipedia/commons/6/68/Comb_filter_feedforward.png" width=400px>
</p>
<p>
    <font size=2><em>Figura 1 - Imagem de <a href="https://commons.wikimedia.org/wiki/File:Comb_filter_feedforward.png">Wikimedia commons</a></em></font>
</p></center>

Dentro do filtro FIR comb temos as mesmas variáveis do filtro FIR comb.

In [3]:
def iircomb(input_signal, gain, delay_length):
    # inicialização do vetor de saída
    y = [0] * len(input_signal)

    # inicialização do vetor de delay
    delay_line = np.zeros(delay_length)

    # aplicação do delay
    for n in range(len(input_signal)):
        y[n] = input_signal[n] + gain * delay_line[delay_length - 1]
        delay_line[1:] = delay_line[:-1]
        delay_line[0] = y[n]
    
    # calcula a resposta em frequência do filtro.
    w, h = freqz(y, input_signal, fs=44100)

    return y, w, h
    

Com todas as funções dos filtros prontas, vamos aplicar o filtro.

Importante notar 

In [4]:

def run_filter(file_name, filter_function, gain, delay_length):
    # leitura do arquivo WAV
    sample_rate, input_signal = read_audio_file(file_name)

    # aplicação do filtro FIR ou IIR
    if filter_function == 'fircomb':
        filtered_data, w, h = fircomb(input_signal, gain, delay_length)
    else:
        filtered_data, w, h = iircomb(input_signal, gain, delay_length)

    # plotagem da resposta em frequência do filtro
    plot_filter(0, sample_rate, w, h)
   
    # escrita do áudio de saída em arquivo
    write_audio_file(filtered_data, sample_rate, 'output_file_comb.wav')

    # plotagem dos sinais de entrada e saída no domínio do tempo e da frequência
    %matplotlib ipympl
    plot_signal(input_signal, sample_rate, 'Sinal de Entrada')
    plot_signal(filtered_data, sample_rate, 'Sinal de Saída')
 
    # mostra os reprodutores de áudio do sinal de entrada e de saída
    display_audio(file_name, 'output_file_comb.wav')


# reseta os widgets
clear_output()
out1.clear_output()
out2.clear_output()
out3.clear_output()

# roda o filtro e apresenta o widget
run_filter('batida.wav', 'fircomb', 0.5, 2000)
widgets_delay()


ValueError: maximum supported dimension for an ndarray is 32, found 374502